In [1]:
import numpy as np
import pandas as pd
import librosa as lr
import os.path
from os import listdir

In [2]:
path = 'Data/Booth'

In [3]:
for group in range(1, 13):
    for trial in range(1, 17):
        filename = f"Data/Booth1/b1_g{group}_t{trial}.wav"
        if not os.path.exists(filename):
            print(filename)

In [4]:
# For musicians in the same trial, the ending points of recordings would generally have a discrepancy of at most a few seconds
# between them. This code ensures that the time series of different musicians in the same trial are right-justified zero-padded 
# where necessary, which allows for transfer entropy calculations to be performed on the time series.

def make_same_length(arrays):
    arr = arrays[:]
    longest = max(arrays, key=len)
    maxLen = len(longest)
    for i, array in enumerate(arr):
        if len(array) < maxLen:
            paddedArray = np.concatenate((array, np.zeros(maxLen - len(array))))
            arr[i] = paddedArray
    return np.array(arr)

In [5]:
sample_rate = 22050

# lr.feature.tonnetz only accepts multiples of 64 samples as window size. 3520 (0.1596 s) is a multiple of 64 and is 
# sufficiently close to the intended window size of 3528 samples (0.16 s).
window_size = 3520
secs_per_window = window_size / sample_rate
print(secs_per_window)

0.15963718820861678


In [6]:
# store the raw time series of the audio files in a DataFrame

raw_timeseries = pd.DataFrame(index=range(0,9_000_000))
for group in range(1,13):
    for trial in range(1,17):
        #print(group, trial)
        trialDict = {}
        for booth in range(1,4):  
            trialDict[booth] = lr.load(f"{path}{booth}/b{booth}_g{group}_t{trial}.wav", mono=True)[0]
            if booth == 3:
                sameLength = make_same_length([trialDict[1],
                                             trialDict[2],
                                             trialDict[3]])
                for booth in range(1,4):
                    timeSeries = sameLength[booth-1]
                    timeSeriesPadded = np.concatenate((timeSeries, np.repeat(np.nan, raw_timeseries.shape[0]-len(sameLength[booth-1]))))
                    raw_timeseries[f"g{group}_t{trial}_b{booth}"] = timeSeriesPadded

        print(f"g{group}_t{trial}")

g1_t1
g1_t2
g1_t3
g1_t4
g1_t5
g1_t6
g1_t7
g1_t8
g1_t9
g1_t10
g1_t11
g1_t12
g1_t13
g1_t14
g1_t15
g1_t16
g2_t1
g2_t2
g2_t3
g2_t4
g2_t5
g2_t6
g2_t7
g2_t8
g2_t9
g2_t10
g2_t11
g2_t12
g2_t13
g2_t14
g2_t15
g2_t16
g3_t1


C:\Users\lenna\AppData\Local\Temp/ipykernel_12448/293267053.py:17: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead.  To get a de-fragmented frame, use `newframe = frame.copy()`
  raw_timeseries[f"g{group}_t{trial}_b{booth}"] = timeSeriesPadded


g3_t2
g3_t3
g3_t4
g3_t5
g3_t6
g3_t7
g3_t8
g3_t9
g3_t10
g3_t11
g3_t12
g3_t13
g3_t14
g3_t15
g3_t16
g4_t1
g4_t2
g4_t3
g4_t4
g4_t5
g4_t6
g4_t7
g4_t8
g4_t9
g4_t10
g4_t11
g4_t12
g4_t13
g4_t14
g4_t15
g4_t16
g5_t1
g5_t2
g5_t3
g5_t4
g5_t5
g5_t6
g5_t7
g5_t8
g5_t9
g5_t10
g5_t11
g5_t12
g5_t13
g5_t14
g5_t15
g5_t16
g6_t1
g6_t2
g6_t3
g6_t4
g6_t5
g6_t6
g6_t7
g6_t8
g6_t9
g6_t10
g6_t11
g6_t12
g6_t13
g6_t14
g6_t15
g6_t16
g7_t1
g7_t2
g7_t3
g7_t4
g7_t5
g7_t6
g7_t7
g7_t8
g7_t9
g7_t10
g7_t11
g7_t12
g7_t13
g7_t14
g7_t15
g7_t16
g8_t1
g8_t2
g8_t3
g8_t4
g8_t5
g8_t6
g8_t7
g8_t8
g8_t9
g8_t10
g8_t11
g8_t12
g8_t13
g8_t14
g8_t15
g8_t16
g9_t1
g9_t2
g9_t3
g9_t4
g9_t5
g9_t6
g9_t7
g9_t8
g9_t9
g9_t10
g9_t11
g9_t12
g9_t13
g9_t14
g9_t15
g9_t16
g10_t1
g10_t2
g10_t3
g10_t4
g10_t5
g10_t6
g10_t7
g10_t8
g10_t9
g10_t10
g10_t11
g10_t12
g10_t13
g10_t14
g10_t15
g10_t16
g11_t1
g11_t2
g11_t3
g11_t4
g11_t5
g11_t6
g11_t7
g11_t8
g11_t9
g11_t10
g11_t11
g11_t12
g11_t13
g11_t14
g11_t15
g11_t16
g12_t1
g12_t2
g12_t3
g12_t4
g12_t5
g12_t6
g12_t

In [7]:
# store the sliding window-averaged time series for each of the three features in separate DataFrames

#rms_timeseries = pd.DataFrame(index=range(0,3000))
#tonal_centroid_timeseries = pd.DataFrame(index=range(0,3000))
spectral_flatness_timeseries = pd.DataFrame(index=range(0,3000))

for col in raw_timeseries.columns:
    print(col)
    """
    rmsTimeSeries = lr.feature.rms(raw_timeseries[col][~np.isnan(raw_timeseries[col])].values, hop_length = window_size)[0] 
    rmsTimeSeriesPadded = np.concatenate((rmsTimeSeries, np.repeat(np.nan, rms_timeseries.shape[0]-len(rmsTimeSeries))))
    rms_timeseries[col] = rmsTimeSeriesPadded

    tonalCentroidTimeSeries = lr.feature.tonnetz(raw_timeseries[col][~np.isnan(raw_timeseries[col])].values, hop_length = window_size)
    for i in range(6):
        tonalCentroidTimeSeriesPadded = np.concatenate((tonalCentroidTimeSeries[i], np.repeat(np.nan, tonal_centroid_timeseries.shape[0]-len(tonalCentroidTimeSeries[i]))))
        tonal_centroid_timeseries[f"{col}-tonnetz{i}"] = tonalCentroidTimeSeriesPadded
    """
    spectralFlatnessTimeSeries = lr.feature.spectral_flatness(raw_timeseries[col][~np.isnan(raw_timeseries[col])].values, hop_length = window_size)[0]
    spectralFlatnessTimeSeriesPadded = np.concatenate((spectralFlatnessTimeSeries, np.repeat(np.nan, spectral_flatness_timeseries.shape[0]-len(spectralFlatnessTimeSeries))))
    spectral_flatness_timeseries[col] = spectralFlatnessTimeSeriesPadded

g1_t1_b1
g1_t1_b2
g1_t1_b3
g1_t2_b1
g1_t2_b2
g1_t2_b3
g1_t3_b1
g1_t3_b2
g1_t3_b3
g1_t4_b1
g1_t4_b2
g1_t4_b3
g1_t5_b1
g1_t5_b2
g1_t5_b3
g1_t6_b1
g1_t6_b2
g1_t6_b3
g1_t7_b1
g1_t7_b2
g1_t7_b3
g1_t8_b1
g1_t8_b2
g1_t8_b3
g1_t9_b1
g1_t9_b2
g1_t9_b3
g1_t10_b1
g1_t10_b2
g1_t10_b3
g1_t11_b1
g1_t11_b2
g1_t11_b3
g1_t12_b1
g1_t12_b2
g1_t12_b3
g1_t13_b1
g1_t13_b2
g1_t13_b3
g1_t14_b1
g1_t14_b2
g1_t14_b3
g1_t15_b1
g1_t15_b2
g1_t15_b3
g1_t16_b1
g1_t16_b2
g1_t16_b3
g2_t1_b1
g2_t1_b2
g2_t1_b3
g2_t2_b1
g2_t2_b2
g2_t2_b3
g2_t3_b1
g2_t3_b2
g2_t3_b3
g2_t4_b1
g2_t4_b2
g2_t4_b3
g2_t5_b1
g2_t5_b2
g2_t5_b3
g2_t6_b1
g2_t6_b2
g2_t6_b3
g2_t7_b1
g2_t7_b2
g2_t7_b3
g2_t8_b1
g2_t8_b2
g2_t8_b3
g2_t9_b1
g2_t9_b2
g2_t9_b3
g2_t10_b1
g2_t10_b2
g2_t10_b3
g2_t11_b1
g2_t11_b2
g2_t11_b3
g2_t12_b1
g2_t12_b2
g2_t12_b3
g2_t13_b1
g2_t13_b2
g2_t13_b3
g2_t14_b1
g2_t14_b2
g2_t14_b3
g2_t15_b1
g2_t15_b2
g2_t15_b3
g2_t16_b1
g2_t16_b2
g2_t16_b3
g3_t1_b1
g3_t1_b2
g3_t1_b3
g3_t2_b1
g3_t2_b2


C:\Users\lenna\AppData\Local\Temp/ipykernel_12448/1590366486.py:21: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead.  To get a de-fragmented frame, use `newframe = frame.copy()`
  spectral_flatness_timeseries[col] = spectralFlatnessTimeSeriesPadded


g3_t2_b3
g3_t3_b1
g3_t3_b2
g3_t3_b3
g3_t4_b1
g3_t4_b2
g3_t4_b3
g3_t5_b1
g3_t5_b2
g3_t5_b3
g3_t6_b1
g3_t6_b2
g3_t6_b3
g3_t7_b1
g3_t7_b2
g3_t7_b3
g3_t8_b1
g3_t8_b2
g3_t8_b3
g3_t9_b1
g3_t9_b2
g3_t9_b3
g3_t10_b1
g3_t10_b2
g3_t10_b3
g3_t11_b1
g3_t11_b2
g3_t11_b3
g3_t12_b1
g3_t12_b2
g3_t12_b3
g3_t13_b1
g3_t13_b2
g3_t13_b3
g3_t14_b1
g3_t14_b2
g3_t14_b3
g3_t15_b1
g3_t15_b2
g3_t15_b3
g3_t16_b1
g3_t16_b2
g3_t16_b3
g4_t1_b1
g4_t1_b2
g4_t1_b3
g4_t2_b1
g4_t2_b2
g4_t2_b3
g4_t3_b1
g4_t3_b2
g4_t3_b3
g4_t4_b1
g4_t4_b2
g4_t4_b3
g4_t5_b1
g4_t5_b2
g4_t5_b3
g4_t6_b1
g4_t6_b2
g4_t6_b3
g4_t7_b1
g4_t7_b2
g4_t7_b3
g4_t8_b1
g4_t8_b2
g4_t8_b3
g4_t9_b1
g4_t9_b2
g4_t9_b3
g4_t10_b1
g4_t10_b2
g4_t10_b3
g4_t11_b1
g4_t11_b2
g4_t11_b3
g4_t12_b1
g4_t12_b2
g4_t12_b3
g4_t13_b1
g4_t13_b2
g4_t13_b3
g4_t14_b1
g4_t14_b2
g4_t14_b3
g4_t15_b1
g4_t15_b2
g4_t15_b3
g4_t16_b1
g4_t16_b2
g4_t16_b3
g5_t1_b1
g5_t1_b2
g5_t1_b3
g5_t2_b1
g5_t2_b2
g5_t2_b3
g5_t3_b1
g5_t3_b2
g5_t3_b3
g5_t4_b1
g5_t4_b2
g5_t4_b3
g5_t5_b1
g5_t5_b2
g5_t5_b3
g5_t

In [13]:
# When a musician has stopped playing but the recording is still going, replace 0.0 values with NaN.

for df in [rms_timeseries, tonal_centroid_timeseries, spectral_flatness_timeseries]:
    for col_index, col_name in enumerate(df.columns):
        done = 0
        
        #if done > 15 and done % 48 == 0:
        #    print(f"Group {done/48} done")
        print(col_name)
        
        ctr = 2999
        while not df[col_name][ctr] > 0:
            df[col_name][ctr] = np.nan
            ctr -= 1
        done += 1

g1_t1_b1
g1_t1_b2
g1_t1_b3
g1_t2_b1
g1_t2_b2
g1_t2_b3
g1_t3_b1
g1_t3_b2
g1_t3_b3
g1_t4_b1
g1_t4_b2
g1_t4_b3
g1_t5_b1
g1_t5_b2
g1_t5_b3
g1_t6_b1
g1_t6_b2
g1_t6_b3
g1_t7_b1
g1_t7_b2
g1_t7_b3
g1_t8_b1
g1_t8_b2
g1_t8_b3
g1_t9_b1
g1_t9_b2
g1_t9_b3
g1_t10_b1
g1_t10_b2
g1_t10_b3
g1_t11_b1
g1_t11_b2
g1_t11_b3
g1_t12_b1
g1_t12_b2
g1_t12_b3
g1_t13_b1
g1_t13_b2
g1_t13_b3
g1_t14_b1
g1_t14_b2
g1_t14_b3
g1_t15_b1
g1_t15_b2
g1_t15_b3
g1_t16_b1
g1_t16_b2
g1_t16_b3
g2_t1_b1
g2_t1_b2
g2_t1_b3
g2_t2_b1
g2_t2_b2
g2_t2_b3
g2_t3_b1
g2_t3_b2
g2_t3_b3
g2_t4_b1
g2_t4_b2
g2_t4_b3
g2_t5_b1
g2_t5_b2
g2_t5_b3
g2_t6_b1
g2_t6_b2
g2_t6_b3
g2_t7_b1
g2_t7_b2
g2_t7_b3
g2_t8_b1
g2_t8_b2
g2_t8_b3
g2_t9_b1
g2_t9_b2
g2_t9_b3
g2_t10_b1
g2_t10_b2
g2_t10_b3
g2_t11_b1
g2_t11_b2
g2_t11_b3
g2_t12_b1
g2_t12_b2
g2_t12_b3
g2_t13_b1
g2_t13_b2
g2_t13_b3
g2_t14_b1
g2_t14_b2
g2_t14_b3
g2_t15_b1
g2_t15_b2
g2_t15_b3
g2_t16_b1
g2_t16_b2
g2_t16_b3
g3_t1_b1
g3_t1_b2
g3_t1_b3
g3_t2_b1
g3_t2_b2
g3_t2_b3
g3_t3_b1
g3_t3_b2
g3_t3_b3
g3_t4_b1
g3_t

g1_t9_b1-tonnetz2
g1_t9_b1-tonnetz3
g1_t9_b1-tonnetz4
g1_t9_b1-tonnetz5
g1_t9_b2-tonnetz0
g1_t9_b2-tonnetz1
g1_t9_b2-tonnetz2
g1_t9_b2-tonnetz3
g1_t9_b2-tonnetz4
g1_t9_b2-tonnetz5
g1_t9_b3-tonnetz0
g1_t9_b3-tonnetz1
g1_t9_b3-tonnetz2
g1_t9_b3-tonnetz3
g1_t9_b3-tonnetz4
g1_t9_b3-tonnetz5
g1_t10_b1-tonnetz0
g1_t10_b1-tonnetz1
g1_t10_b1-tonnetz2
g1_t10_b1-tonnetz3
g1_t10_b1-tonnetz4
g1_t10_b1-tonnetz5
g1_t10_b2-tonnetz0
g1_t10_b2-tonnetz1
g1_t10_b2-tonnetz2
g1_t10_b2-tonnetz3
g1_t10_b2-tonnetz4
g1_t10_b2-tonnetz5
g1_t10_b3-tonnetz0
g1_t10_b3-tonnetz1
g1_t10_b3-tonnetz2
g1_t10_b3-tonnetz3
g1_t10_b3-tonnetz4
g1_t10_b3-tonnetz5
g1_t11_b1-tonnetz0
g1_t11_b1-tonnetz1
g1_t11_b1-tonnetz2
g1_t11_b1-tonnetz3
g1_t11_b1-tonnetz4
g1_t11_b1-tonnetz5
g1_t11_b2-tonnetz0
g1_t11_b2-tonnetz1
g1_t11_b2-tonnetz2
g1_t11_b2-tonnetz3
g1_t11_b2-tonnetz4
g1_t11_b2-tonnetz5
g1_t11_b3-tonnetz0
g1_t11_b3-tonnetz1
g1_t11_b3-tonnetz2
g1_t11_b3-tonnetz3
g1_t11_b3-tonnetz4
g1_t11_b3-tonnetz5
g1_t12_b1-tonnetz0
g1_t12_b1

g3_t1_b3-tonnetz0
g3_t1_b3-tonnetz1
g3_t1_b3-tonnetz2
g3_t1_b3-tonnetz3
g3_t1_b3-tonnetz4
g3_t1_b3-tonnetz5
g3_t2_b1-tonnetz0
g3_t2_b1-tonnetz1
g3_t2_b1-tonnetz2
g3_t2_b1-tonnetz3
g3_t2_b1-tonnetz4
g3_t2_b1-tonnetz5
g3_t2_b2-tonnetz0
g3_t2_b2-tonnetz1
g3_t2_b2-tonnetz2
g3_t2_b2-tonnetz3
g3_t2_b2-tonnetz4
g3_t2_b2-tonnetz5
g3_t2_b3-tonnetz0
g3_t2_b3-tonnetz1
g3_t2_b3-tonnetz2
g3_t2_b3-tonnetz3
g3_t2_b3-tonnetz4
g3_t2_b3-tonnetz5
g3_t3_b1-tonnetz0
g3_t3_b1-tonnetz1
g3_t3_b1-tonnetz2
g3_t3_b1-tonnetz3
g3_t3_b1-tonnetz4
g3_t3_b1-tonnetz5
g3_t3_b2-tonnetz0
g3_t3_b2-tonnetz1
g3_t3_b2-tonnetz2
g3_t3_b2-tonnetz3
g3_t3_b2-tonnetz4
g3_t3_b2-tonnetz5
g3_t3_b3-tonnetz0
g3_t3_b3-tonnetz1
g3_t3_b3-tonnetz2
g3_t3_b3-tonnetz3
g3_t3_b3-tonnetz4
g3_t3_b3-tonnetz5
g3_t4_b1-tonnetz0
g3_t4_b1-tonnetz1
g3_t4_b1-tonnetz2
g3_t4_b1-tonnetz3
g3_t4_b1-tonnetz4
g3_t4_b1-tonnetz5
g3_t4_b2-tonnetz0
g3_t4_b2-tonnetz1
g3_t4_b2-tonnetz2
g3_t4_b2-tonnetz3
g3_t4_b2-tonnetz4
g3_t4_b2-tonnetz5
g3_t4_b3-tonnetz0
g3_t4_b3-t

g4_t10_b2-tonnetz4
g4_t10_b2-tonnetz5
g4_t10_b3-tonnetz0
g4_t10_b3-tonnetz1
g4_t10_b3-tonnetz2
g4_t10_b3-tonnetz3
g4_t10_b3-tonnetz4
g4_t10_b3-tonnetz5
g4_t11_b1-tonnetz0
g4_t11_b1-tonnetz1
g4_t11_b1-tonnetz2
g4_t11_b1-tonnetz3
g4_t11_b1-tonnetz4
g4_t11_b1-tonnetz5
g4_t11_b2-tonnetz0
g4_t11_b2-tonnetz1
g4_t11_b2-tonnetz2
g4_t11_b2-tonnetz3
g4_t11_b2-tonnetz4
g4_t11_b2-tonnetz5
g4_t11_b3-tonnetz0
g4_t11_b3-tonnetz1
g4_t11_b3-tonnetz2
g4_t11_b3-tonnetz3
g4_t11_b3-tonnetz4
g4_t11_b3-tonnetz5
g4_t12_b1-tonnetz0
g4_t12_b1-tonnetz1
g4_t12_b1-tonnetz2
g4_t12_b1-tonnetz3
g4_t12_b1-tonnetz4
g4_t12_b1-tonnetz5
g4_t12_b2-tonnetz0
g4_t12_b2-tonnetz1
g4_t12_b2-tonnetz2
g4_t12_b2-tonnetz3
g4_t12_b2-tonnetz4
g4_t12_b2-tonnetz5
g4_t12_b3-tonnetz0
g4_t12_b3-tonnetz1
g4_t12_b3-tonnetz2
g4_t12_b3-tonnetz3
g4_t12_b3-tonnetz4
g4_t12_b3-tonnetz5
g4_t13_b1-tonnetz0
g4_t13_b1-tonnetz1
g4_t13_b1-tonnetz2
g4_t13_b1-tonnetz3
g4_t13_b1-tonnetz4
g4_t13_b1-tonnetz5
g4_t13_b2-tonnetz0
g4_t13_b2-tonnetz1
g4_t13_b2-to

g6_t3_b1-tonnetz2
g6_t3_b1-tonnetz3
g6_t3_b1-tonnetz4
g6_t3_b1-tonnetz5
g6_t3_b2-tonnetz0
g6_t3_b2-tonnetz1
g6_t3_b2-tonnetz2
g6_t3_b2-tonnetz3
g6_t3_b2-tonnetz4
g6_t3_b2-tonnetz5
g6_t3_b3-tonnetz0
g6_t3_b3-tonnetz1
g6_t3_b3-tonnetz2
g6_t3_b3-tonnetz3
g6_t3_b3-tonnetz4
g6_t3_b3-tonnetz5
g6_t4_b1-tonnetz0
g6_t4_b1-tonnetz1
g6_t4_b1-tonnetz2
g6_t4_b1-tonnetz3
g6_t4_b1-tonnetz4
g6_t4_b1-tonnetz5
g6_t4_b2-tonnetz0
g6_t4_b2-tonnetz1
g6_t4_b2-tonnetz2
g6_t4_b2-tonnetz3
g6_t4_b2-tonnetz4
g6_t4_b2-tonnetz5
g6_t4_b3-tonnetz0
g6_t4_b3-tonnetz1
g6_t4_b3-tonnetz2
g6_t4_b3-tonnetz3
g6_t4_b3-tonnetz4
g6_t4_b3-tonnetz5
g6_t5_b1-tonnetz0
g6_t5_b1-tonnetz1
g6_t5_b1-tonnetz2
g6_t5_b1-tonnetz3
g6_t5_b1-tonnetz4
g6_t5_b1-tonnetz5
g6_t5_b2-tonnetz0
g6_t5_b2-tonnetz1
g6_t5_b2-tonnetz2
g6_t5_b2-tonnetz3
g6_t5_b2-tonnetz4
g6_t5_b2-tonnetz5
g6_t5_b3-tonnetz0
g6_t5_b3-tonnetz1
g6_t5_b3-tonnetz2
g6_t5_b3-tonnetz3
g6_t5_b3-tonnetz4
g6_t5_b3-tonnetz5
g6_t6_b1-tonnetz0
g6_t6_b1-tonnetz1
g6_t6_b1-tonnetz2
g6_t6_b1-t

g7_t11_b3-tonnetz5
g7_t12_b1-tonnetz0
g7_t12_b1-tonnetz1
g7_t12_b1-tonnetz2
g7_t12_b1-tonnetz3
g7_t12_b1-tonnetz4
g7_t12_b1-tonnetz5
g7_t12_b2-tonnetz0
g7_t12_b2-tonnetz1
g7_t12_b2-tonnetz2
g7_t12_b2-tonnetz3
g7_t12_b2-tonnetz4
g7_t12_b2-tonnetz5
g7_t12_b3-tonnetz0
g7_t12_b3-tonnetz1
g7_t12_b3-tonnetz2
g7_t12_b3-tonnetz3
g7_t12_b3-tonnetz4
g7_t12_b3-tonnetz5
g7_t13_b1-tonnetz0
g7_t13_b1-tonnetz1
g7_t13_b1-tonnetz2
g7_t13_b1-tonnetz3
g7_t13_b1-tonnetz4
g7_t13_b1-tonnetz5
g7_t13_b2-tonnetz0
g7_t13_b2-tonnetz1
g7_t13_b2-tonnetz2
g7_t13_b2-tonnetz3
g7_t13_b2-tonnetz4
g7_t13_b2-tonnetz5
g7_t13_b3-tonnetz0
g7_t13_b3-tonnetz1
g7_t13_b3-tonnetz2
g7_t13_b3-tonnetz3
g7_t13_b3-tonnetz4
g7_t13_b3-tonnetz5
g7_t14_b1-tonnetz0
g7_t14_b1-tonnetz1
g7_t14_b1-tonnetz2
g7_t14_b1-tonnetz3
g7_t14_b1-tonnetz4
g7_t14_b1-tonnetz5
g7_t14_b2-tonnetz0
g7_t14_b2-tonnetz1
g7_t14_b2-tonnetz2
g7_t14_b2-tonnetz3
g7_t14_b2-tonnetz4
g7_t14_b2-tonnetz5
g7_t14_b3-tonnetz0
g7_t14_b3-tonnetz1
g7_t14_b3-tonnetz2
g7_t14_b3-to

g9_t4_b2-tonnetz5
g9_t4_b3-tonnetz0
g9_t4_b3-tonnetz1
g9_t4_b3-tonnetz2
g9_t4_b3-tonnetz3
g9_t4_b3-tonnetz4
g9_t4_b3-tonnetz5
g9_t5_b1-tonnetz0
g9_t5_b1-tonnetz1
g9_t5_b1-tonnetz2
g9_t5_b1-tonnetz3
g9_t5_b1-tonnetz4
g9_t5_b1-tonnetz5
g9_t5_b2-tonnetz0
g9_t5_b2-tonnetz1
g9_t5_b2-tonnetz2
g9_t5_b2-tonnetz3
g9_t5_b2-tonnetz4
g9_t5_b2-tonnetz5
g9_t5_b3-tonnetz0
g9_t5_b3-tonnetz1
g9_t5_b3-tonnetz2
g9_t5_b3-tonnetz3
g9_t5_b3-tonnetz4
g9_t5_b3-tonnetz5
g9_t6_b1-tonnetz0
g9_t6_b1-tonnetz1
g9_t6_b1-tonnetz2
g9_t6_b1-tonnetz3
g9_t6_b1-tonnetz4
g9_t6_b1-tonnetz5
g9_t6_b2-tonnetz0
g9_t6_b2-tonnetz1
g9_t6_b2-tonnetz2
g9_t6_b2-tonnetz3
g9_t6_b2-tonnetz4
g9_t6_b2-tonnetz5
g9_t6_b3-tonnetz0
g9_t6_b3-tonnetz1
g9_t6_b3-tonnetz2
g9_t6_b3-tonnetz3
g9_t6_b3-tonnetz4
g9_t6_b3-tonnetz5
g9_t7_b1-tonnetz0
g9_t7_b1-tonnetz1
g9_t7_b1-tonnetz2
g9_t7_b1-tonnetz3
g9_t7_b1-tonnetz4
g9_t7_b1-tonnetz5
g9_t7_b2-tonnetz0
g9_t7_b2-tonnetz1
g9_t7_b2-tonnetz2
g9_t7_b2-tonnetz3
g9_t7_b2-tonnetz4
g9_t7_b2-tonnetz5
g9_t7_b3-t

g10_t12_b3-tonnetz1
g10_t12_b3-tonnetz2
g10_t12_b3-tonnetz3
g10_t12_b3-tonnetz4
g10_t12_b3-tonnetz5
g10_t13_b1-tonnetz0
g10_t13_b1-tonnetz1
g10_t13_b1-tonnetz2
g10_t13_b1-tonnetz3
g10_t13_b1-tonnetz4
g10_t13_b1-tonnetz5
g10_t13_b2-tonnetz0
g10_t13_b2-tonnetz1
g10_t13_b2-tonnetz2
g10_t13_b2-tonnetz3
g10_t13_b2-tonnetz4
g10_t13_b2-tonnetz5
g10_t13_b3-tonnetz0
g10_t13_b3-tonnetz1
g10_t13_b3-tonnetz2
g10_t13_b3-tonnetz3
g10_t13_b3-tonnetz4
g10_t13_b3-tonnetz5
g10_t14_b1-tonnetz0
g10_t14_b1-tonnetz1
g10_t14_b1-tonnetz2
g10_t14_b1-tonnetz3
g10_t14_b1-tonnetz4
g10_t14_b1-tonnetz5
g10_t14_b2-tonnetz0
g10_t14_b2-tonnetz1
g10_t14_b2-tonnetz2
g10_t14_b2-tonnetz3
g10_t14_b2-tonnetz4
g10_t14_b2-tonnetz5
g10_t14_b3-tonnetz0
g10_t14_b3-tonnetz1
g10_t14_b3-tonnetz2
g10_t14_b3-tonnetz3
g10_t14_b3-tonnetz4
g10_t14_b3-tonnetz5
g10_t15_b1-tonnetz0
g10_t15_b1-tonnetz1
g10_t15_b1-tonnetz2
g10_t15_b1-tonnetz3
g10_t15_b1-tonnetz4
g10_t15_b1-tonnetz5
g10_t15_b2-tonnetz0
g10_t15_b2-tonnetz1
g10_t15_b2-tonnetz2


g12_t4_b1-tonnetz2
g12_t4_b1-tonnetz3
g12_t4_b1-tonnetz4
g12_t4_b1-tonnetz5
g12_t4_b2-tonnetz0
g12_t4_b2-tonnetz1
g12_t4_b2-tonnetz2
g12_t4_b2-tonnetz3
g12_t4_b2-tonnetz4
g12_t4_b2-tonnetz5
g12_t4_b3-tonnetz0
g12_t4_b3-tonnetz1
g12_t4_b3-tonnetz2
g12_t4_b3-tonnetz3
g12_t4_b3-tonnetz4
g12_t4_b3-tonnetz5
g12_t5_b1-tonnetz0
g12_t5_b1-tonnetz1
g12_t5_b1-tonnetz2
g12_t5_b1-tonnetz3
g12_t5_b1-tonnetz4
g12_t5_b1-tonnetz5
g12_t5_b2-tonnetz0
g12_t5_b2-tonnetz1
g12_t5_b2-tonnetz2
g12_t5_b2-tonnetz3
g12_t5_b2-tonnetz4
g12_t5_b2-tonnetz5
g12_t5_b3-tonnetz0
g12_t5_b3-tonnetz1
g12_t5_b3-tonnetz2
g12_t5_b3-tonnetz3
g12_t5_b3-tonnetz4
g12_t5_b3-tonnetz5
g12_t6_b1-tonnetz0
g12_t6_b1-tonnetz1
g12_t6_b1-tonnetz2
g12_t6_b1-tonnetz3
g12_t6_b1-tonnetz4
g12_t6_b1-tonnetz5
g12_t6_b2-tonnetz0
g12_t6_b2-tonnetz1
g12_t6_b2-tonnetz2
g12_t6_b2-tonnetz3
g12_t6_b2-tonnetz4
g12_t6_b2-tonnetz5
g12_t6_b3-tonnetz0
g12_t6_b3-tonnetz1
g12_t6_b3-tonnetz2
g12_t6_b3-tonnetz3
g12_t6_b3-tonnetz4
g12_t6_b3-tonnetz5
g12_t7_b1-to

g9_t2_b2
g9_t2_b3
g9_t3_b1
g9_t3_b2
g9_t3_b3
g9_t4_b1
g9_t4_b2
g9_t4_b3
g9_t5_b1
g9_t5_b2
g9_t5_b3
g9_t6_b1
g9_t6_b2
g9_t6_b3
g9_t7_b1
g9_t7_b2
g9_t7_b3
g9_t8_b1
g9_t8_b2
g9_t8_b3
g9_t9_b1
g9_t9_b2
g9_t9_b3
g9_t10_b1
g9_t10_b2
g9_t10_b3
g9_t11_b1
g9_t11_b2
g9_t11_b3
g9_t12_b1
g9_t12_b2
g9_t12_b3
g9_t13_b1
g9_t13_b2
g9_t13_b3
g9_t14_b1
g9_t14_b2
g9_t14_b3
g9_t15_b1
g9_t15_b2
g9_t15_b3
g9_t16_b1
g9_t16_b2
g9_t16_b3
g10_t1_b1
g10_t1_b2
g10_t1_b3
g10_t2_b1
g10_t2_b2
g10_t2_b3
g10_t3_b1
g10_t3_b2
g10_t3_b3
g10_t4_b1
g10_t4_b2
g10_t4_b3
g10_t5_b1
g10_t5_b2
g10_t5_b3
g10_t6_b1
g10_t6_b2
g10_t6_b3
g10_t7_b1
g10_t7_b2
g10_t7_b3
g10_t8_b1
g10_t8_b2
g10_t8_b3
g10_t9_b1
g10_t9_b2
g10_t9_b3
g10_t10_b1
g10_t10_b2
g10_t10_b3
g10_t11_b1
g10_t11_b2
g10_t11_b3
g10_t12_b1
g10_t12_b2
g10_t12_b3
g10_t13_b1
g10_t13_b2
g10_t13_b3
g10_t14_b1
g10_t14_b2
g10_t14_b3
g10_t15_b1
g10_t15_b2
g10_t15_b3
g10_t16_b1
g10_t16_b2
g10_t16_b3
g11_t1_b1
g11_t1_b2
g11_t1_b3
g11_t2_b1
g11_t2_b2
g11_t2_b3
g11_t3_b1
g11_t3_b2
g1

In [9]:
# Export

#rms_timeseries.to_csv("rms_timeseries.csv", index=False)
#tonal_centroid_timeseries.to_csv("tonal_centroid_timeseries.csv", index=False)
spectral_flatness_timeseries.to_csv("spectral_flatness_timeseries.csv", index=False)

In [ ]:
endingDataset = pd.read_csv("Data/endingDataset_fromThomas_withconditions.csv", delimiter=";")

In [100]:
postTrialQuest = pd.read_csv("Data/postTrialQuestExp2_Results_withComs.csv")

In [103]:
postTrialQuest.replace({
    "X": 0,
    "MOI": 1,
    "NOUS": 2
}, inplace=True)

postTrialQuest[(postTrialQuest["prompt_ecoute_Louise"] != postTrialQuest["prompt_heard"])]

,trio,booth,musician,take,prompt_ecoute_Louise,prompt_heard,goodEnd,likedImpro,guessPromptA,guessPromptB,guessPromptC
275,8,c,24,12,1,0,1.0,3.0,2.0,2.0,0.0


In [77]:
# prompt_ecoute_Louise (the prompt given) and prompt_heard (the prompt heard by the musician) are identical in all cases
# except one. Drop prompt_ecoute_Louise from df, proceed with prompt_heard.
postTrialQuest.drop(columns=["prompt_ecoute_Louise"], inplace=True)

In [104]:
# Further preprocessing
postTrialQuest.replace({
    "a": 1,
    "b": 2,
    "c": 3
}, inplace=True)

In [105]:
postTrialQuest

,trio,booth,musician,take,prompt_ecoute_Louise,prompt_heard,goodEnd,likedImpro,guessPromptA,guessPromptB,guessPromptC
0,1,1,1,5,0,0,5.0,4.0,0.0,0.0,0.0
1,1,2,2,5,1,1,5.0,5.0,2.0,1.0,2.0
2,1,3,3,5,0,0,6.0,4.0,0.0,0.0,0.0
3,1,1,1,6,0,0,7.0,5.0,0.0,0.0,1.0
4,1,2,2,6,0,0,1.0,5.0,2.0,0.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...
427,12,2,35,15,1,1,4.0,4.0,2.0,1.0,2.0
428,12,3,36,15,0,0,6.0,6.0,1.0,0.0,0.0
429,12,1,34,16,2,2,6.0,6.0,2.0,2.0,2.0
430,12,2,35,16,2,2,7.0,6.0,2.0,2.0,2.0


In [107]:
for group in range(1, 13):
    for trial in range(1,17):
        dataDict[f"g{group}_t{trial}"]["prompt_time"] = endingDataset[(endingDataset["trio"] == group) & (endingDataset["take"] == trial)]["promptTime"]
        dataDict[f"g{group}_t{trial}"]["prompt_type"] = endingDataset[(endingDataset["trio"] == group) & (endingDataset["take"] == trial)]["promptTime"]
        for booth in range(1, 4):
            dataDict[f"g{group}_t{trial}"][f"b{booth}"]["prompt_heard"] = postTrialQuest[(postTrialQuest["trio"] == group) & (postTrialQuest["take"] == trial)]["prompt_heard"]

In [96]:
endingDataset.columns

Index(['trioTake', 'trio', 'take', 'startA', 'startB', 'startC', 'endA',
       'endB', 'endC', 'startAll', 'endAll', 'promptTime', 'syncIndex',
       'duration', 'type', 'number', 'pedalA', 'pedalB', 'pedalC',
       'numberPedals', 'goalAchievementAll', 'goodEndAll', 'goodImproAll',
       'goodEndA', 'goodEndB', 'goodEndC'],
      dtype='object')

In [57]:
endingDataset.columns

Index(['trioTake', 'trio', 'take', 'startA', 'startB', 'startC', 'endA',
       'endB', 'endC', 'startAll', 'endAll', 'promptTime', 'syncIndex',
       'duration', 'type', 'number', 'pedalA', 'pedalB', 'pedalC',
       'numberPedals', 'goalAchievementAll', 'goodEndAll', 'goodImproAll',
       'goodEndA', 'goodEndB', 'goodEndC'],
      dtype='object')